Great. I was thinking just this morning how much I feel like implementing a concurrency system. Doesn't look too bad, but it'll be a bit fiddly, I expect.

In [1]:
from collections import defaultdict
import re

So the state of the whole system requires:

* a location for each of the two programs
* a queue for each program
* a set of registers for each program (a ddict with initial values for p)

Let's try:

In [2]:
state=[{'loc':0, 'queue':[], 'registers':defaultdict(int)},
       {'loc':0, 'queue':[], 'registers':defaultdict(int)}]

state[0]['registers']['p']=0
state[1]['registers']['p']=1

state

[{'loc': 0, 'queue': [], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 0, 'queue': [], 'registers': defaultdict(int, {'p': 1})}]

Now, redefine the function `apply_command` so that it takes 3 parameters: the command, the program which called the command (0 or 1), and the state:

In [3]:
def apply_command(command_str, program_i, stateIn):
    '''
    Updates stateIn with the command command_str called by program_i.
    Returns True if the program has resulted in a changed state,
    False otherwise.
    '''
    #if program_i:
    #    print('\t'+command_str)
    #else:
    #    print(command_str)
    
    # Useful to have a local function to check for ints:
    # (Could use exceptions, but I prefer not to if possible)
    def is_int(stringIn_str):
        '''
        True if stringIn_str can be converted to int
        '''
        if re.match('^\-?\d+$', stringIn_str):
            return True
        else:
            return False
    
    command_ls=command_str.strip().split()
    if command_ls[0]=='set':
        # If second argument is non-integral, treat it
        # as a ddict index
        if is_int(command_ls[2]):
            stateIn[program_i]['registers'][command_ls[1]]=int(command_ls[2])
        else:
            stateIn[program_i]['registers'][command_ls[1]]=stateIn[program_i]['registers'][command_ls[2]]
        # Advance the command line
        stateIn[program_i]['loc']+=1
            
    elif command_ls[0]=='snd':
        # Add value to the end of the other program's queue
        if program_i==0:
            me_prog=0
            them_prog=1
        else:
            me_prog=1
            them_prog=0
        
        if is_int(command_ls[1]):
            stateIn[them_prog]['queue'].append(int(command_ls[1]))
        else:
            stateIn[them_prog]['queue'].append(stateIn[me_prog]['registers'][command_ls[1]])
        # Advance the command line
        stateIn[me_prog]['loc']+=1
            
    elif command_ls[0]=='add':
        v=stateIn[program_i]['registers'][command_ls[1]]
        if is_int(command_ls[2]):
            stateIn[program_i]['registers'][command_ls[1]]=v+int(command_ls[2])
        else:
            stateIn[program_i]['registers'][command_ls[1]]=v+stateIn[program_i]['registers'][command_ls[2]]
        # Advance the command line
        stateIn[program_i]['loc']+=1

    elif command_ls[0]=='mul':
        v=stateIn[program_i]['registers'][command_ls[1]]
        if is_int(command_ls[2]):
            stateIn[program_i]['registers'][command_ls[1]]=v*int(command_ls[2])
        else:
            stateIn[program_i]['registers'][command_ls[1]]=v*stateIn[program_i]['registers'][command_ls[2]]
        # Advance the command line
        stateIn[program_i]['loc']+=1

    elif command_ls[0]=='mod':
        v=stateIn[program_i]['registers'][command_ls[1]]
        if is_int(command_ls[2]):
            stateIn[program_i]['registers'][command_ls[1]]=v%int(command_ls[2])
        else:
            stateIn[program_i]['registers'][command_ls[1]]=v%stateIn[program_i]['registers'][command_ls[2]]
        # Advance the command line
        stateIn[program_i]['loc']+=1


    elif command_ls[0]=='jgz':
        if is_int(command_ls[1]):
            v=int(command_ls[1])
        else:
            v=stateIn[program_i]['registers'][command_ls[1]]
        # Jump if v>0
        if v>0:
            if is_int(command_ls[2]):
                stateIn[program_i]['loc']+=int(command_ls[2])
            else:
                stateIn[program_i]['loc']+=stateIn[program_i]['registers'][command_ls[2]]
        # else just go forward 1:
        else:
            stateIn[program_i]['loc']+=1
    
    # So for recovery, obtain the next value from the
    # queue if it exists. If not, the process is
    # locked, and return True
    elif command_ls[0]=='rcv':
        
        if len(stateIn[program_i]['queue'])==0:
            return False
            
        else:
            stateIn[program_i]['registers'][command_ls[1]]=stateIn[program_i]['queue'].pop(0)
            stateIn[program_i]['loc']+=1

    else:
        raise ValueError("Command not recognised")
            
    return True

Set the test input:

So... a little test

In [4]:
state=[{'loc':0, 'queue':[], 'registers':defaultdict(int)},
       {'loc':0, 'queue':[], 'registers':defaultdict(int)}]

state[0]['registers']['p']=0
state[1]['registers']['p']=1

state

[{'loc': 0, 'queue': [], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 0, 'queue': [], 'registers': defaultdict(int, {'p': 1})}]

In [5]:
print(apply_command('snd 1', 0, state))
state

True


[{'loc': 1, 'queue': [], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 0, 'queue': [1], 'registers': defaultdict(int, {'p': 1})}]

In [6]:
print(apply_command('snd 2', 0, state))
state

True


[{'loc': 2, 'queue': [], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 0, 'queue': [1, 2], 'registers': defaultdict(int, {'p': 1})}]

In [7]:
print(apply_command('snd p', 0, state))
state

True


[{'loc': 3, 'queue': [], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 0, 'queue': [1, 2, 0], 'registers': defaultdict(int, {'p': 1})}]

In [8]:
print(apply_command('rcv a', 0, state))
state

False


[{'loc': 3, 'queue': [], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 0, 'queue': [1, 2, 0], 'registers': defaultdict(int, {'p': 1})}]

In [9]:
print(apply_command('snd 1', 1, state))
state

True


[{'loc': 3, 'queue': [1], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 1, 'queue': [1, 2, 0], 'registers': defaultdict(int, {'p': 1})}]

In [10]:
print(apply_command('snd 2', 1, state))
state

True


[{'loc': 3, 'queue': [1, 2], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 2, 'queue': [1, 2, 0], 'registers': defaultdict(int, {'p': 1})}]

In [11]:
print(apply_command('snd p', 1, state))
state

True


[{'loc': 3, 'queue': [1, 2, 1], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 3, 'queue': [1, 2, 0], 'registers': defaultdict(int, {'p': 1})}]

In [12]:
print(apply_command('rcv a', 1, state))
state

True


[{'loc': 3, 'queue': [1, 2, 1], 'registers': defaultdict(int, {'p': 0})},
 {'loc': 4, 'queue': [2, 0], 'registers': defaultdict(int, {'a': 1, 'p': 1})}]

In [13]:
print(apply_command('rcv a', 0, state))
state

True


[{'loc': 4, 'queue': [2, 1], 'registers': defaultdict(int, {'a': 1, 'p': 0})},
 {'loc': 4, 'queue': [2, 0], 'registers': defaultdict(int, {'a': 1, 'p': 1})}]

Those seem to be working OK on the face of it. Let's try with the test input:

In [14]:
testInput_str='''
snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
'''

Now we need to write a cell that will keep calling each of the programs until both are locked. I'm going to define a function `run_to_lock` which will keep calling `apply_command` until the program is locked.

In [15]:
def run_to_lock(code_ls, stateIn, program_i):
    '''
    Calls apply_command on program_i in state stateIn on the 
    code code_ls until the program hits a deadlock, or
    leaves the code. Returns the number of steps until
    termination.
    '''

    
    # Otherwise, run until get False
    i=0
    while True:
        tv=apply_command(code_ls[stateIn[program_i]['loc']], program_i, stateIn)
        
        if not tv:
            return i
        else:
            i+=1
            # Can finish if the program has 
            # exited the code
            if stateIn[program_i]['loc']<0 or stateIn[program_i]['loc']>=len(code_ls):
                return i


In [16]:
# Initial setup.

state=[{'loc':0, 'queue':[], 'registers':defaultdict(int)},
       {'loc':0, 'queue':[], 'registers':defaultdict(int)}]

state[0]['registers']['p']=0
state[1]['registers']['p']=1

# Parse the code

code_ls=testInput_str.strip().split('\n')

# Now run program 0 until it loks

run_to_lock(code_ls, state, 0)


3

In [17]:
run_to_lock(code_ls, state, 1)


6

In [18]:
run_to_lock(code_ls, state, 0)


3

Hmmm... that might actually be working. So let's see if we can alternate the programs until they're both locked:

In [19]:
# Initial setup.

state=[{'loc':0, 'queue':[], 'registers':defaultdict(int)},
       {'loc':0, 'queue':[], 'registers':defaultdict(int)}]

state[0]['registers']['p']=0
state[1]['registers']['p']=1

# Parse the code

code_ls=testInput_str.strip().split('\n')

s=[True, True]
prog_i=0

while not s==[False, False]:
    c=run_to_lock(code_ls, state, prog_i)
    if c>0:
        s[prog_i]=True
    else:
        s[prog_i]=False
    if prog_i==0:
        prog_i=1
    else:
        prog_i=0


Finally, we'll need a quick redefinition to count the `snd` commands. Easiest to put that in `run_to_lock`:

In [20]:
def run_to_lock_snd(code_ls, stateIn, program_i):
    '''
    Calls apply_command on program_i in state stateIn on the 
    code code_ls until the program hits a deadlock, or
    leaves the code. Returns the pair of the number of steps
    until termination plus the number of calls to snd.
    '''
    
    # Run until get False
    i=0
    callsToSnd_i=0
    while True:
        command_str=code_ls[stateIn[program_i]['loc']]
        tv=apply_command(command_str, program_i, stateIn)
        
        if not tv:
            return (i, callsToSnd_i)
        else:
            i+=1
            if command_str[0:3]=='snd':
                callsToSnd_i+=1
                
            # Can finish if the program has 
            # exited the code
            if stateIn[program_i]['loc']<0 or stateIn[program_i]['loc']>=len(code_ls):
                return (i, callsToSnd_i)


In [21]:
# Initial setup.

state=[{'loc':0, 'queue':[], 'registers':defaultdict(int)},
       {'loc':0, 'queue':[], 'registers':defaultdict(int)}]

state[0]['registers']['p']=0
state[1]['registers']['p']=1

# Parse the code
with open('data/day18.txt') as fIn:
    code_ls=fIn.read().strip().split('\n')

s=[True, True]
prog_i=0

callsToSnd_ls=[0, 0]

while not s==[False, False]:
    (totalCalls_i, callsToSnd_i)=run_to_lock_snd(code_ls, state, prog_i)
    if totalCalls_i>0:
        s[prog_i]=True
    else:
        s[prog_i]=False
    callsToSnd_ls[prog_i]+=callsToSnd_i
    if prog_i==0:
        prog_i=1
    else:
        prog_i=0

callsToSnd_ls[1]


8001

That was a bit of a slog. Wonder how quickly others got it?